In [1]:
!pip install git+https://github.com/openai/whisper.git 
!pip install datasets==1.18.3
!pip install jiwer
!pip install pydub
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-t_iullbd
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-t_iullbd
     |████████████████████████████████| 5.5 MB 4.4 MB/s 
     |████████████████████████████████| 7.6 MB 16.1 MB/s 
     |████████████████████████████████| 163 kB 55.9 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175246 sha256=4b391211bb701c7f2e097b480674cb47d6549214a36343004d1932787bb4605a
  Stored in directory: /tmp/pip-ephem-wheel-cache-jugaow53/wheels/16/15/89/1c7bb31bd0006793a95549d04785121a8a36daad9158e1e43a
Successfully built whisper
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 311 kB 4.6 MB/s 
     |████████████████████████████████| 115 kB 52.9 MB/s 
     |███████████████████████████

In [2]:
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [3]:
audio, sr = get_audio()

In [4]:
import scipy
scipy.io.wavfile.write('recording.wav', sr, audio)

In [5]:
import whisper
from datasets import load_metric
from pydub import AudioSegment

In [6]:
def read_text(filename):
  full_path = 'Rekaman_'+filename+'.txt'
  txt = open(full_path, "r")
  strings = txt.read()
  return strings

def split_audio(filename):
  audio = AudioSegment.from_wav(filename+'.wav')
  audio_split = audio.split_to_mono()
  audio_split[0].export(filename+'left.wav', format='wav')
  audio_split[1].export(filename+'right.wav', format='wav')

#Show stats
def check_audio_stats(filename):
    audio_segment = AudioSegment.from_file(filename+".wav")
    channels = audio_segment.channels
    sampling_rate = audio_segment.frame_rate
    return channels, sampling_rate

def split_audio(filename):
    audio = AudioSegment.from_wav(filename+".wav")
    audio_split = audio.split_to_mono()
    audio_split[0].export(filename+'_left.wav', format="wav")
    audio_split[1].export(filename+'_right.wav', format="wav")

In [7]:
model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:42<00:00, 35.6MiB/s]


In [8]:
print(model.transcribe('recording.wav'))

/usr/local/lib/python3.7/dist-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


{'text': ' to?', 'segments': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 18.6, 'text': ' to?', 'tokens': [50364, 281, 30, 51294], 'temperature': 1.0, 'avg_logprob': -4.024372863769531, 'compression_ratio': 0.2727272727272727, 'no_speech_prob': 0.5092400312423706}], 'language': 'en'}
